In [3]:
# download packages
!pip install transformers==4.8.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 65.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 98.7 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895258 sha256=4d08935b41e7bc2e927d37f851c75cd5d8f353638c8246a3da16fafd4584b8ce
  Stored in directory: /root/.cache/pip/wheels/12/1c/3d/46cf06718d63a32ff798a89594b61e7f345ab6b36d909ce033
Successfully built sacremoses


In [ ]:
# import packages
import re
import torch
import random
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel

# mount google drive
from google.colab import drive 
drive.mount('/content/gdrive')

project_folder = '/content/gdrive/MyDrive/266_Final_Project_Folder'

Mounted at /content/gdrive


In [ ]:
## Define class and functions
#--------

# Dataset class
class SentimentDataset(Dataset):
    def __init__(self, txt_list, label_list, tokenizer, max_length):
        # define variables    
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        map_label = {0:'negative', 4: 'positive'}
        # iterate through the dataset
        for txt, label in zip(txt_list, label_list):
            # prepare the text
            prep_txt = f'<|startoftext|>Tweet: {txt}<|pad|>Sentiment: {map_label[label]}<|endoftext|>'
            # tokenize
            encodings_dict = tokenizer(prep_txt, truncation=True,
                                       max_length=max_length, padding="max_length")
            # append to list
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
            self.labels.append(map_label[label])

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx], self.labels[idx]

# Data load function
def load_sentiment_dataset(tokenizer):
    # load dataset and sample 10k reviews.
    file_path = project_folder + "/datasets/training.1600000.processed.noemoticon.csv"
    df = pd.read_csv(file_path, encoding='ISO-8859-1', header=None)
    df = df[[0, 5]]
    df.columns = ['label', 'text']
    df = df.sample(10000, random_state=1)

    # divide into test and train
    X_train, X_test, y_train, y_test = \
              train_test_split(df['text'].tolist(), df['label'].tolist(),
              shuffle=True, test_size=0.05, random_state=1, stratify=df['label'])

    print('X_train type:', type(X_train))
    print('X_train[:10]', X_train[:10])
    print('y_train type:', type(y_train))
    print('y_train[:10]', y_train[:10])

    # format into SentimentDataset class
    train_dataset = SentimentDataset(X_train, y_train, tokenizer, max_length=512)

    # return
    return train_dataset, (X_test, y_test)

In [ ]:
## Load model and data
#--------

# set model name
model_name = "gpt2"
# seed
torch.manual_seed(42)

# load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained(model_name, bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>', pad_token='<|pad|>')
model = GPT2LMHeadModel .from_pretrained(model_name).cuda()
model.resize_token_embeddings(len(tokenizer))

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Embedding(50259, 768)

In [ ]:
# prepare and load dataset
train_dataset, test_dataset = load_sentiment_dataset(tokenizer)

X_train type: <class 'list'>
X_train[:10] ['Tuesdays are... you guessed it, Tuesdays. At the library getting stuff done. I forgot how boring school is ', 'Just watched slumdog millionaire. Now i understand. ', 'just got home from night at the museum 2 and it was awesome  when the jonas brothers came in I screamed so loud and everyone got mad ', 'Follow?  â\x99¥, D-Day, Red Wings, Pens, Honduras, Penguins, Detroit, Jay-Z, #phish, Palm Pre, Pittsburgh', "have had a super amazing day today! wow. I've even surprised myself! ", "@bcbeatty I guess I'll try again when I get back home. Right now only have all, @ mentions and DMs ", 'Awww the bears are waking up ', '2 and 1/2 more hours. thank goodness! ', 'Been dealing with MIDI sync issues with the Virus Snow ALL DAY.  nerd talk ', 'fun fun . . . just The Hills now . . . 3 hours (about) to finale!  (and 4 to the MTV Movie Awards!)']
y_train type: <class 'list'>
y_train[:10] [0, 4, 0, 4, 4, 0, 4, 4, 0, 4]


In [ ]:
len(train_dataset.input_ids)

9500

In [ ]:
train_dataset.input_ids

[tensor([50257, 47845,    25, 48496, 12545,   389,   986,   345, 25183,   340,
            11, 48496, 12545,    13,  1629,   262,  5888,  1972,  3404,  1760,
            13,   314, 16453,   703, 14262,  1524,   318, 50258, 31837,  3681,
            25,  4633, 50256, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
         50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
         50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
         50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
         50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
         50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
         50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
         50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
         50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
         50258, 50258, 50258, 50258, 50258, 50258, 5

In [ ]:
train_dataset.labels

['negative',
 'positive',
 'negative',
 'positive',
 'positive',
 'negative',
 'positive',
 'positive',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'positive',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'positive',

In [ ]:
train_dataset.attn_masks

[tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0

In [ ]:
test_dataset[0]

['@lisasamples Delaware...too far to drive ',
 '@angelicahondm lucky you ',
 "@GCUnderhill My pleasure Thank you am losing 5 people an hour, guess they don't love me  I did remove allot they didn't have their DM on",
 'I feel so bored and feel like my life is meaningless in many ways ',
 "don't think i can make it to shibuya tonight. i still have to clean and prepare the beats for soi. ",
 "@Ryanwmitchell I recognize that sink! Sad I couldn't see you guys. ",
 "On my way to work,btw it's 8 at morning in Germany.seems like sun is on vacation ",
 'just got out of school - I love my friends...have THE most amazing letters/notes in my yearbook&lt;3 ',
 "@zacmartin I'm hyper connected. Leave me alone. ",
 'I just realized my daddy wont be home for fathers day ',
 '@MitchAllan Yeah, I just got home.  Cool! I want to check out the new music!',
 '@bayushi_akai Hmm.... tempting offer, but no ',
 'Alias bom dia twitters ',
 "I may have also bought something from Amazon that I've been wanting for

In [ ]:
test_dataset[1]

[0,
 4,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 4,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 0,
 4,
 0,
 4,
 4,
 0,
 0,
 0,
 4,
 4,
 4,
 0,
 0,
 0,
 0,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 4,
 4,
 0,
 4,
 0,
 4,
 0,
 0,
 0,
 0,
 4,
 4,
 4,
 4,
 4,
 0,
 0,
 4,
 0,
 0,
 4,
 0,
 4,
 0,
 4,
 0,
 4,
 4,
 4,
 4,
 0,
 0,
 4,
 4,
 4,
 0,
 4,
 4,
 4,
 0,
 4,
 4,
 0,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 4,
 4,
 4,
 4,
 0,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 0,
 0,
 4,
 4,
 0,
 4,
 4,
 0,
 0,
 4,
 0,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 4,
 4,
 4,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 4,
 0,
 4,
 0,
 4,
 4,
 4,
 0,
 4,
 4,
 0,
 0,
 4,
 0,
 0,
 4,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 4,
 4,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 4,
 4,
 0,
 4,
 4,
 0,
 4,
 4,
 0,
 0,
 0,
 0,
 4,
 4,
 4,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 4,
 4,
 0,
 4,
 4,
 0,
 4,
 0,
 4,
 4,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 4,
 0,
 4,
 4,
 4,
 4,


In [ ]:
## Train
#--------
# creating training arguments
training_args = TrainingArguments(output_dir='results', num_train_epochs=2, logging_steps=10,
                                 load_best_model_at_end=True, save_strategy="epoch", evaluation_strategy="epoch",
                                 per_device_train_batch_size=2, per_device_eval_batch_size=2,
                                 warmup_steps=100, weight_decay=0.01, logging_dir='logs')

# start training
Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=test_dataset,
        data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                    'attention_mask': torch.stack([f[1] for f in data]),
                                    'labels': torch.stack([f[0] for f in data])}).train()

***** Running training *****
  Num examples = 9500
  Num Epochs = 2
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 9500


***** Running Evaluation *****
  Num examples = 2
  Batch size = 2


TypeError: ignored

In [4]:
# import
import transformers
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# model name
model_name = "gpt2"

# load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).cuda()

# create prompt
prompt = """
Below are some examples for sentiment detection of movie reviews.

Review: I am sad that the hero died.
Sentiment: Negative

Review: The ending was perfect.
Sentiment: Positive

Review: The plot was not so good!
Sentiment:"""

# generate tokens
generated = tokenizer(prompt, return_tensors="pt").input_ids.cuda()

# perform prediction 
sample_outputs = model.generate(generated, do_sample=False, top_k=50, max_length=512, top_p=0.90, 
        temperature=0, num_return_sequences=0)

# decode the predicted tokens into texts
predicted_text = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
print(predicted_text)
"""Output --> 
Below are some examples for sentiment detection of movie reviews.

Review: I am sad that the hero died.
Sentiment: Negative

Review: The ending was perfect.
Sentiment: Positive

Review: The plot was not so good!
Sentiment: Negative
"""

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Below are some examples for sentiment detection of movie reviews.

Review: I am sad that the hero died.
Sentiment: Negative

Review: The ending was perfect.
Sentiment: Positive

Review: The plot was not so good!
Sentiment: Negative

Review: The ending was not so good!

Sentiment: Negative

Review: The ending was not so good!

Sentiment: Negative

Review: The ending was not so good!

Sentiment: Negative

Review: The ending was not so good!

Sentiment: Negative

Review: The ending was not so good!

Sentiment: Negative

Review: The ending was not so good!

Sentiment: Negative

Review: The ending was not so good!

Sentiment: Negative

Review: The ending was not so good!

Sentiment: Negative

Review: The ending was not so good!

Sentiment: Negative

Review: The ending was not so good!

Sentiment: Negative

Review: The ending was not so good!

Sentiment: Negative

Review: The ending was not so good!

Sentiment: Negative

Review: The ending was not so good!

Sentiment: Negative

Review: The 

'Output --> \nBelow are some examples for sentiment detection of movie reviews.\n\nReview: I am sad that the hero died.\nSentiment: Negative\n\nReview: The ending was perfect.\nSentiment: Positive\n\nReview: The plot was not so good!\nSentiment: Negative\n'